# Dashboard Principal - Gestión de Matrimonio

## Presupuesto Total: $10,000,000 COP

Este notebook proporciona una vista general del estado del presupuesto, invitados y hospedaje.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

## 1. Carga de Datos

In [ ]:
# Cargar datos
df_presupuesto = pd.read_csv('../datos/presupuesto.csv')
df_invitados = pd.read_csv('../datos/invitados.csv')
df_hospedaje = pd.read_csv('../datos/hospedaje.csv')

# Constante de presupuesto total
PRESUPUESTO_TOTAL = 10_000_000

print("✓ Datos cargados exitosamente")

## 2. Resumen Ejecutivo - KPIs Principales

In [ ]:
# Calcular métricas principales
total_gastado = df_presupuesto['abonado'].sum()
total_comprometido = df_presupuesto['valor_total'].sum()
saldo_disponible = PRESUPUESTO_TOTAL - total_comprometido
porcentaje_usado = (total_comprometido / PRESUPUESTO_TOTAL) * 100

# Invitados
total_invitados = df_invitados['personas'].sum()
invitados_confirmados = df_invitados[df_invitados['confirmado'] == True]['personas'].sum()
total_hospedaje = df_invitados['hospedaje'].sum()

# Determinar color de alerta
if porcentaje_usado < 70:
    color_alerta = '🟢 VERDE'
    estado = 'Presupuesto saludable'
elif porcentaje_usado < 90:
    color_alerta = '🟡 AMARILLO'
    estado = 'Precaución - monitorear gastos'
else:
    color_alerta = '🔴 ROJO'
    estado = 'ALERTA - presupuesto comprometido'

print("="*60)
print("           RESUMEN EJECUTIVO - MATRIMONIO")
print("="*60)
print(f"\n💰 PRESUPUESTO")
print(f"   Presupuesto Total:     ${PRESUPUESTO_TOTAL:>12,}")
print(f"   Comprometido:          ${total_comprometido:>12,.0f}")
print(f"   Abonado:               ${total_gastado:>12,.0f}")
print(f"   Saldo por pagar:       ${(total_comprometido - total_gastado):>12,.0f}")
print(f"   Disponible:            ${saldo_disponible:>12,.0f}")
print(f"   Porcentaje usado:      {porcentaje_usado:>12.1f}%")
print(f"\n🚦 ESTADO: {color_alerta} - {estado}")
print(f"\n👥 INVITADOS")
print(f"   Total personas:        {total_invitados:>12}")
print(f"   Confirmados:           {invitados_confirmados:>12}")
print(f"   Pendientes:            {total_invitados - invitados_confirmados:>12}")
print(f"   Requieren hospedaje:   {total_hospedaje:>12}")
print("="*60)

## 3. Visualizaciones Principales

In [ ]:
# Crear figura con múltiples subgráficos
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Gráfico de pastel - Distribución por categoría
gastos_por_categoria = df_presupuesto.groupby('categoria')['valor_total'].sum()
colors_pie = ['#FF6B6B', '#4ECDC4']
axes[0, 0].pie(gastos_por_categoria, labels=gastos_por_categoria.index, autopct='%1.1f%%',
               startangle=90, colors=colors_pie)
axes[0, 0].set_title('Distribución del Presupuesto por Categoría', fontsize=14, fontweight='bold')

# 2. Barra de progreso del presupuesto
categorias_prog = ['Comprometido', 'Disponible']
valores_prog = [total_comprometido, saldo_disponible]
colors_bar = ['#dc3545' if porcentaje_usado > 90 else '#ffc107' if porcentaje_usado > 70 else '#28a745', '#e0e0e0']
axes[0, 1].barh(categorias_prog, valores_prog, color=colors_bar)
axes[0, 1].set_xlabel('Pesos COP', fontsize=12)
axes[0, 1].set_title(f'Progreso del Presupuesto ({porcentaje_usado:.1f}% usado)', fontsize=14, fontweight='bold')
axes[0, 1].axvline(x=PRESUPUESTO_TOTAL, color='red', linestyle='--', linewidth=2, label='Límite')
for i, v in enumerate(valores_prog):
    axes[0, 1].text(v, i, f' ${v:,.0f}', va='center', fontsize=10)

# 3. Gráfico de barras - Top 10 gastos
top_gastos = df_presupuesto.nlargest(10, 'valor_total')[['item', 'valor_total', 'abonado', 'saldo']]
x = range(len(top_gastos))
width = 0.25
axes[1, 0].bar([i - width for i in x], top_gastos['valor_total'], width, label='Total', color='#3498db')
axes[1, 0].bar(x, top_gastos['abonado'], width, label='Abonado', color='#2ecc71')
axes[1, 0].bar([i + width for i in x], top_gastos['saldo'], width, label='Saldo', color='#e74c3c')
axes[1, 0].set_xlabel('Ítem', fontsize=12)
axes[1, 0].set_ylabel('Valor (COP)', fontsize=12)
axes[1, 0].set_title('Top 10 Gastos - Detalle', fontsize=14, fontweight='bold')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(top_gastos['item'], rotation=45, ha='right')
axes[1, 0].legend()

# 4. Invitados por grupo
invitados_por_grupo = df_invitados.groupby('grupo')['personas'].sum()
colors_groups = ['#FF6B6B', '#4ECDC4', '#95E1D3']
axes[1, 1].bar(invitados_por_grupo.index, invitados_por_grupo.values, color=colors_groups)
axes[1, 1].set_ylabel('Número de Personas', fontsize=12)
axes[1, 1].set_title('Invitados por Grupo de Origen', fontsize=14, fontweight='bold')
for i, v in enumerate(invitados_por_grupo.values):
    axes[1, 1].text(i, v, str(v), ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## 4. Tabla Resumen por Categoría

In [ ]:
# Resumen por categoría
resumen_categoria = df_presupuesto.groupby('categoria').agg({
    'valor_total': 'sum',
    'abonado': 'sum',
    'saldo': 'sum',
    'item': 'count'
}).rename(columns={'item': 'num_items'})

resumen_categoria['%_del_total'] = (resumen_categoria['valor_total'] / total_comprometido * 100).round(1)

# Formatear para mostrar
resumen_display = resumen_categoria.copy()
resumen_display['valor_total'] = resumen_display['valor_total'].apply(lambda x: f'${x:,.0f}')
resumen_display['abonado'] = resumen_display['abonado'].apply(lambda x: f'${x:,.0f}')
resumen_display['saldo'] = resumen_display['saldo'].apply(lambda x: f'${x:,.0f}')
resumen_display['%_del_total'] = resumen_display['%_del_total'].apply(lambda x: f'{x}%')

print("\n📊 RESUMEN POR CATEGORÍA\n")
print(resumen_display.to_string())

## 5. Alertas y Advertencias

In [ ]:
print("\n⚠️  ALERTAS Y PENDIENTES\n")
print("="*60)

# Alerta de presupuesto
if porcentaje_usado > 90:
    print("🔴 CRÍTICO: Has usado más del 90% del presupuesto!")
elif porcentaje_usado > 70:
    print("🟡 ADVERTENCIA: Has usado más del 70% del presupuesto.")
else:
    print("🟢 Presupuesto en buen estado.")

# Items sin confirmar
items_sin_confirmar = df_presupuesto[df_presupuesto['confirmado'] == False]
if len(items_sin_confirmar) > 0:
    print(f"\n⚠️  {len(items_sin_confirmar)} ítems sin confirmar:")
    for idx, row in items_sin_confirmar.iterrows():
        if row['valor_total'] > 0:
            print(f"   - {row['item']}: ${row['valor_total']:,.0f}")

# Invitados sin confirmar
invitados_sin_confirmar = df_invitados[df_invitados['confirmado'] == False]
if len(invitados_sin_confirmar) > 0:
    total_personas_sin_confirmar = invitados_sin_confirmar['personas'].sum()
    print(f"\n👥 {len(invitados_sin_confirmar)} grupos sin confirmar ({total_personas_sin_confirmar} personas)")

# Hospedajes sin asignar
hospedajes_sin_asignar = df_hospedaje[df_hospedaje['asignado'] == False]
if len(hospedajes_sin_asignar) > 0:
    total_personas_hospedaje = hospedajes_sin_asignar['personas_hospedaje'].sum()
    print(f"\n🏨 {len(hospedajes_sin_asignar)} hospedajes sin asignar ({total_personas_hospedaje} personas)")

print("="*60)

## 6. Próximos Pasos Recomendados

In [ ]:
print("\n✅ PRÓXIMOS PASOS RECOMENDADOS:\n")
print("1. Actualizar valores en datos/presupuesto.csv con cotizaciones reales")
print("2. Confirmar asistencia de invitados en datos/invitados.csv")
print("3. Asignar habitaciones en datos/hospedaje.csv")
print("4. Revisar notebooks adicionales:")
print("   - 02_analisis_presupuesto.ipynb")
print("   - 03_gestion_invitados.ipynb")
print("   - 04_hospedaje.ipynb")
print("\n💡 TIP: Recarga este notebook después de actualizar los CSV para ver cambios en tiempo real.")